In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from darts import TimeSeries
from darts.models import BlockRNNModel
from darts.metrics import mse, rmse, r2_score, mae, smape
from darts.dataprocessing.transformers import Scaler
import torch
from optuna.integration import PyTorchLightningPruningCallback
from pytorch_lightning.callbacks import Callback, EarlyStopping
import optuna
import os
import json

# Visualization settings
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (12, 6)  # Set the default figure size
torch.set_float32_matmul_precision('medium')  # Set precision for matrix multiplication

In [2]:
# Define target columns for time series forecasting
target_columns = [
    'Temperature','Precipitation_accumulated','Humidity', 'Wind_Speed_kmh',
    'Soil_Moisture', 'Soil_Temperature', 'Wind_Dir_Sin', 'Wind_Dir_Cos'
]

# Load the cleaned ground station data
df = pd.read_csv("../data/ground_station_clean.csv")

In [3]:
# Configure encoders for time features and data transformation
encoders = {
    "cyclic":{  # Cyclic features for time components
        "past":["month","dayofyear","day","hour","minute"],
    },
    "transformer": Scaler()  # Data scaling transformer
}

In [4]:
# Custom callback class that combines PyTorch Lightning pruning with a standard callback
class PatchedPruningCallback(PyTorchLightningPruningCallback, Callback):
    pass

# Create a directory to save iteration results
results_output_dir = "optuna_iteration_metrics"
os.makedirs(results_output_dir, exist_ok=True)  # Create if it doesn't exist

In [5]:
def objective(trial):
    """
    Objective function for Optuna optimization that trains and evaluates a model with given hyperparameters.
    """
    # Define hyperparameters to be optimized
    input_chunk_length = trial.suggest_int("input_chunk_length", 12, 72)  # Input sequence length
    hidden_size = trial.suggest_int("hidden_size", 32, 256)  # Hidden layer dimension
    layer_size = trial.suggest_int("layer_size", 1, 4)  # Number of RNN layers
    dropout = trial.suggest_float("dropout", 0.1, 0.4)  # Dropout rate
    batch_size = trial.suggest_int("batch_size", 16, 64)  # Training batch size
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)  # Learning rate (log scale)

    # Configure early stopping
    early_stopper = EarlyStopping("val_loss", min_delta=0.001, patience=10, verbose=True)
    pl_trainer_kwargs = {
            "accelerator": "auto",
            "callbacks": [early_stopper],
    }

    # Split data into training and validation sets (80/20)
    n = int(len(df)*0.8)
    train_df_fold, val_df_fold = df.iloc[:n], df.iloc[n:]

    # Convert dataframes to TimeSeries objects
    train_fold = TimeSeries.from_dataframe(train_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')
    val_fold = TimeSeries.from_dataframe(val_df_fold, time_col="Timestamp", value_cols=target_columns, freq='1h')

    # Print trial information
    print(f"\nStarting Trial {trial.number}")
    print(f"Hyperparameters: {trial.params}")
    print("\nTraining the model...")
    print(f"Train set: {len(train_fold)} samples")  # Use len() for TimeSeries objects
    print(f"Validation set: {len(val_fold)} samples")  # Use len() for TimeSeries objects

    # Scale the data using only training data statistics
    scaler = Scaler()
    scaler = scaler.fit(train_fold)
    train_scaled = scaler.transform(train_fold)
    val_scaled = scaler.transform(val_fold)

    # Define working directory and model name for consistency
    _work_dir = "/home/eduardo/Documentos/Water-Cycle-Neural-Network/darts_logs/"  # Ensure this path exists and is writable
    _model_name = "model_optuna_temp"  # Temporary model name for each trial

    # Create the working directory if it doesn't exist
    os.makedirs(_work_dir, exist_ok=True)

    # Configure the RNN model with trial hyperparameters
    model = BlockRNNModel(
        model="GRU",  # Use GRU architecture
        model_name=_model_name,
        work_dir=_work_dir,
        input_chunk_length=input_chunk_length,
        output_chunk_length=1,  # Predict one step ahead
        hidden_dim=hidden_size,
        n_rnn_layers=layer_size,
        n_epochs=30,  # Maximum number of training epochs
        batch_size=batch_size,
        dropout=dropout,
        add_encoders=encoders,
        pl_trainer_kwargs=pl_trainer_kwargs,
        loss_fn=torch.nn.HuberLoss(),  # Huber loss is robust to outliers
        lr_scheduler_cls=torch.optim.lr_scheduler.ReduceLROnPlateau,
        lr_scheduler_kwargs={"mode":"min", "factor":0.5, "patience":4, "min_lr":1e-6},
        save_checkpoints=True,
        show_warnings=True,
        force_reset=True,  # Important to train from scratch for each trial with this model_name
        optimizer_kwargs={"lr": lr, "weight_decay": 1e-5},
    )

    # Train the model
    model.fit(
        series=train_scaled,
        val_series=val_scaled,
        dataloader_kwargs={"num_workers": 11}
    )

    # Try to load the best model from checkpoint
    try:
        loaded_model = BlockRNNModel.load_from_checkpoint(model_name=_model_name, work_dir=_work_dir, best=True)
        print(f"Model loaded from checkpoint for trial {trial.number}")
    except FileNotFoundError:
        print(f"Checkpoint not found for {_model_name} in {_work_dir}. Using in-memory trained model.")
        loaded_model = model  # Use the trained model if loading fails (fallback)

    # Generate historical forecasts to evaluate model performance
    forecasts = loaded_model.historical_forecasts(
        train_scaled,  # Use training data
        start=0.8,  # Start forecasting at 80% of the training data
        forecast_horizon=1,  # Forecast one step ahead
        stride=1,  # Generate a forecast at each possible time point
        retrain=False,  # Don't retrain the model for each forecast
    )

    # Inverse transform forecasts and actual values back to an original scale
    forecasts_t = scaler.inverse_transform(forecasts)
    s = scaler.inverse_transform(train_scaled).split_after(0.8)[1]  # Get the actual values for comparison

    # Calculate performance metrics for each target variable
    metrics = {}
    print("Starting time series consistency verification...")
    try:
        for target in target_columns:
            # Calculate multiple error metrics for comprehensive evaluation
            metrics[target] = {
                'MSE': mse(s[target], forecasts_t[target]),  # Mean Squared Error
                'RMSE': rmse(s[target], forecasts_t[target]),  # Root Mean Squared Error
                'MAE': mae(s[target], forecasts_t[target]),  # Mean Absolute Error
                'R2': r2_score(s[target], forecasts_t[target]),  # R-squared score
                'SMAPE': smape(s[target], forecasts_t[target]),  # Symmetric Mean Absolute Percentage Error
            }
        # Create a DataFrame for better visualization of metrics
        metrics_df = pd.DataFrame(metrics).T
        print("\nPerformance metrics:")
        print(metrics_df)
    except Exception as e:
        print(e)

    # Calculate overall SMAPE across all variables (optimization target)
    overall_smape_val = smape(s, forecasts_t)
    print(f"The SMAPE for this fold was {overall_smape_val}")

    # Store trial results in a dictionary
    trial_dict = {
        "trial_number": trial.number,
        "input_chunk_length": input_chunk_length,
        "hidden_size": hidden_size,
        "layer_size": layer_size,
        "dropout": dropout,
        "batch_size": batch_size,
        "lr": lr,
        "overall_smape_val": overall_smape_val,
        "metrics": metrics_df.to_dict(),
    }

    # Save trial results to a JSON file
    json_path = os.path.join(results_output_dir, f"trial_{trial.number}.json")
    with open(json_path, 'w') as f:
        json.dump(trial_dict, f, indent=4)
    print(f"Results for trial {trial.number} saved to {json_path}")

    # Return the optimization metric (SMAPE), or infinity if NaN (to be minimized)
    return overall_smape_val if not np.isnan(overall_smape_val) else float("inf")

In [6]:
# Callback function to print trial progress information
def print_callback(study, trial):
    """Print current trial results and best results so far."""
    print(f"Current value: {trial.value}, Current params: {trial.params}")
    print(f"Best value: {study.best_value}, Best params: {study.best_trial.params}")

In [7]:
# Create and run the optimization study
study = optuna.create_study(direction="minimize")  # We want to minimize SMAPE
study.optimize(objective, n_trials=50, callbacks=[print_callback])

[I 2025-05-18 13:38:57,860] A new study created in memory with name: no-name-dd0652da-52bd-45b5-bd55-2379b51ec1d0



Starting Trial 0
Hyperparameters: {'input_chunk_length': 56, 'hidden_size': 157, 'layer_size': 3, 'dropout': 0.32594151509381697, 'batch_size': 52, 'lr': 2.8303964894204734e-05}

Training the model...
Train set: 6647 samples
Validation set: 1662 samples


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | HuberLoss        | 0      | train
1 | train_criterion | HuberLoss        | 0      | train
2 | val_criterion   | HuberLoss        | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | GRU              | 381 K  | train
6 | fc              | Sequential       | 1.3 K  | train
-------------------------------------------------------------
382 K     Trainable params
0         Non-trainable params
382 K     Total params
1.529     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 10 records. Best score: 0.013. Signaling Trainer to stop.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Model loaded from checkpoint for trial 49


/home/eduardo/Documentos/Water-Cycle-Neural-Network/venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
[I 2025-05-18 15:15:39,846] Trial 49 finished with value: 45.147420764017916 and parameters: {'input_chunk_length': 43, 'hidden_size': 115, 'layer_size': 3, 'dropout': 0.21448430209787864, 'batch_size': 46, 'lr': 0.002045653072635162}. Best is trial 9 with value: 43.28550537729951.


Starting time series consistency verification...

Performance metrics:
                                    MSE        RMSE        MAE         R2  \
Temperature                    1.092953    1.045444   0.740422   0.678954   
Precipitation_accumulated     60.887871    7.803068   6.915880 -63.150328   
Humidity                       5.352995    2.313654   1.812399   0.917683   
Wind_Speed_kmh                 0.085121    0.291755   0.132845   0.749872   
Soil_Moisture              28956.554867  170.166257  77.032925   0.960868   
Soil_Temperature               0.521881    0.722413   0.524371   0.907010   
Wind_Dir_Sin                   0.055967    0.236574   0.121240   0.651553   
Wind_Dir_Cos                   0.046909    0.216585   0.119193   0.836607   

                                SMAPE  
Temperature                 13.102892  
Precipitation_accumulated    1.517720  
Humidity                     2.149196  
Wind_Speed_kmh             154.503965  
Soil_Moisture                1.2371

In [8]:
# Print the best results from the optimization
print(f"Best Value (Minimum): {study.best_value}")
print(f"Best Parameters: {study.best_params}")

Best Value (Minimum): 43.28550537729951
Best Parameters: {'input_chunk_length': 49, 'hidden_size': 216, 'layer_size': 3, 'dropout': 0.27524990279731704, 'batch_size': 45, 'lr': 0.0023425422126866643}


In [9]:
# Save the best trial results to a separate file
best_dict = {
    "best_value": study.best_value,
    "best_params": study.best_params,
}
json_path = os.path.join(results_output_dir, "best_trial.json")
with open(json_path, 'w') as f:
    json.dump(best_dict, f, indent=4)
print(f"Best results saved to {json_path}")


Best results saved to optuna_iteration_metrics/best_trial.json
